In [2]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor as SKKNNRegressor
from sklearn.linear_model import LinearRegression as SKLinearRegression
import folium
from IPython.display import display

# Haversine_distance
def haversine_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    r = 6371.0
    return r * c

# KNN Regressor
class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = []
        for x in X:
            distances = np.linalg.norm(self.X_train - x, axis=1)
            nearest_neighbors_idx = distances.argsort()[:self.n_neighbors]
            nearest_neighbors_labels = self.y_train[nearest_neighbors_idx]
            predictions.append(np.mean(nearest_neighbors_labels))
        return np.array(predictions)

# Linear Regression
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None
        self.velocity = None  # For Momentum
        self.cache = None  # For AdaGrad

    def _initialize_parameters(self, n_features):
        self.weights = np.zeros(n_features)
        self.bias = 0
        if self.optimization == 'Momentum':
            self.velocity = np.zeros(n_features)
        if self.optimization == 'AdaGrad':
            self.cache = np.zeros(n_features)

    def _mse(self, y_true, y_pred):
        return np.mean((y_true - y_pred) ** 2)

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._initialize_parameters(n_features)

        for i in range(self.max_iter):
            y_pred = np.dot(X, self.weights) + self.bias

            dw = -(2 / n_samples) * np.dot(X.T, (y - y_pred))
            db = -(2 / n_samples) * np.sum(y - y_pred)

            if self.optimization == 'SGD':
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db

            elif self.optimization == 'Momentum':
                beta = 0.9
                self.velocity = beta * self.velocity + (1 - beta) * dw
                self.weights -= self.learning_rate * self.velocity
                self.bias -= self.learning_rate * db

            elif self.optimization == 'AdaGrad':
                self.cache += dw ** 2
                self.weights -= (self.learning_rate / (np.sqrt(self.cache) + self.epsilon)) * dw
                self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

df = pd.read_csv('input_data.csv', delimiter=';')

# Добавление признаков is_Moscow и is_Saint_Peterburg
moscow_center = (55.751244, 37.618423)
peterburg_center = (59.939099, 30.315877)

df['is_Moscow'] = df.apply(lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], moscow_center[0], moscow_center[1]) <= 20, axis=1)
df['is_Saint_Peterburg'] = df.apply(lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], peterburg_center[0], peterburg_center[1]) <= 20, axis=1)

# Удаление ненужных признаков
df = df.drop(['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'], axis=1)

# Анализ признаков
categorical_columns = df.select_dtypes(include=['bool', 'object', 'category']).columns.tolist()
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# OneHot-Encoding категориальных признаков
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_categorical = encoder.fit_transform(df[categorical_columns])

# Преобразование закодированных признаков в DataFrame
encoded_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_columns))

# Объединение закодированных категориальных признаков с числовыми признаками
df_encoded = pd.concat([df[numerical_columns], encoded_df], axis=1)

# Разделение данных для обучения и тестирования
X = df_encoded.drop(columns=['price']).values
y = df_encoded['price'].values

# Разбиение и масштабирование данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение собственных моделей
knn_regressor = KNNRegressor(n_neighbors=5)
linear_regressor = LinearRegression(learning_rate=0.01, optimization='AdaGrad', max_iter=1000)

knn_regressor.fit(X_train_scaled, y_train)
linear_regressor.fit(X_train_scaled, y_train)

# Предсказания на тестовых данных
knn_predictions = knn_regressor.predict(X_test_scaled)
linear_predictions = linear_regressor.predict(X_test_scaled)

# Использование библиотечных моделей для сравнения
sk_knn = SKKNNRegressor(n_neighbors=5)
sk_linear = SKLinearRegression()

sk_knn.fit(X_train_scaled, y_train)
sk_linear.fit(X_train_scaled, y_train)

sk_knn_predictions = sk_knn.predict(X_test_scaled)
sk_linear_predictions = sk_linear.predict(X_test_scaled)

# Вычисление метрик для собственных моделей
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mse, mae, rmse

# Метрики для собственных моделей
mse_knn, mae_knn, rmse_knn = calculate_metrics(y_test, knn_predictions)
mse_linear, mae_linear, rmse_linear = calculate_metrics(y_test, linear_predictions)

# Метрики для библиотечных моделей
mse_sk_knn, mae_sk_knn, rmse_sk_knn = calculate_metrics(y_test, sk_knn_predictions)
mse_sk_linear, mae_sk_linear, rmse_sk_linear = calculate_metrics(y_test, sk_linear_predictions)

# Вывод результатов
print("KNN Regressor: MSE =", mse_knn, ", MAE =", mae_knn, ", RMSE =", rmse_knn)
print("Linear Regression: MSE =", mse_linear, ", MAE =", mae_linear, ", RMSE =", rmse_linear)

print("Sklearn KNN Regressor Metrics: MSE =", mse_sk_knn, ", MAE =", mae_sk_knn, ", RMSE =", rmse_sk_knn)
print("Sklearn Linear Regression Metrics: MSE =", mse_sk_linear, ", MAE =", mae_sk_linear, ", RMSE =", rmse_sk_linear)

# Отображение карты с точками
map_df_full = pd.read_csv('input_data.csv', delimiter=';').loc[:1000]
lats = map_df_full['geo_lat'].loc[:1000]
longs = map_df_full['geo_lon'].loc[:1000]

# Инициализация карты с центром в Москве
m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Добавление точек на карту
for point in zip(lats, longs):
    folium.Marker(location=[point[0], point[1]]).add_to(m)

# Отображение карты
display(m)


Our KNN Regressor Metrics: MSE = 1467630363725476.5 , MAE = 2466187.309589662 , RMSE = 38309664.10353237
Our Linear Regression Metrics: MSE = 1649546994433170.8 , MAE = 14396360.50470302 , RMSE = 40614615.52733413
Sklearn KNN Regressor Metrics: MSE = 1467629815962724.8 , MAE = 2465876.970921182 , RMSE = 38309656.95438586
Sklearn Linear Regression Metrics: MSE = 5032208443715059.0 , MAE = 42737834.30021928 , RMSE = 70938060.61427855


# Часть 1. Подготовим данные для обработки моделями машинного обучения.
---

*   Категориальные признаки - is_Moscow и is_Saint_Peterburg
*   Числовые признаки - price, area, rooms

# Часть 2. Эксперименты с моделями машинного обучения.
---
###Эксперименты показали, что библиотечные функции выдают более лучший результат по сравнению с собственными реализациями. Результат эксперимента:
* KNN Regressor (из кода): MSE = 1467630363725476.5 , MAE = 2466187.309589662 , RMSE = 38309664.10353237
* KNN Regressor (Sklearn): MSE = 1467629815962724.8 , MAE = 2465876.970921182 , RMSE = 38309656.95438586
* Linear Regression (из кода): MSE = 1649546994433170.8 , MAE = 14396360.50470302 , RMSE = 40614615.52733413
* Linear Regression (Sklearn): MSE = 5032208443715059.0 , MAE = 42737834.30021928 , RMSE = 70938060.61427855